In [3]:
import os,sys
from typing import List, Dict
from tqdm import tqdm
import numpy as np
import pandas as pd
from sklearn.metrics import accuracy_score, roc_auc_score
import lightgbm as lgb
import matplotlib.pyplot as plt


# Code 경로 추가
sys.path.append(os.path.dirname(os.path.dirname(os.path.abspath(""))))
print(sys.path[-1])

C:\Users\Minari\Documents\GitHub\level1-classificationinmachinelearning-recsys-06


In [3]:
'''# 파일 호출
data_path: str = "../../data"
file_name = 'raw.csv'
if not os.path.exists(f'{data_path}/{file_name}'):
    from Code.dataset.merge_all import merge_all
    df = merge_all(data_path)

df: pd.DataFrame = pd.read_csv(os.path.join(data_path,"raw.csv"))
submission_df: pd.DataFrame = pd.read_csv(os.path.join(data_path, "test.csv")) # ID, target 열만 가진 데이터 미리 호출
'''

In [4]:
'''
csv 파일 합치기
기준은 datetime 으로
겹치는 컬럼은 _dup.숫자 형식으로 -> _dup.1
'''
merged_df = None
if not os.path.exists('merged_df.csv'):
    import pandas as pd
    import os
    
    data_dir = '../../data'
    file_names = [name for name in os.listdir(data_dir) if name.endswith('.csv')][:-4]
    
    files = [file for file in os.listdir(data_dir) if file.endswith('.csv')]
    
    df_list = []
    for name in file_names:
        df = pd.read_csv(os.path.join(data_dir, name)) 
        df_list.append(df)
    
    merged_df = df_list[0]
    for df in df_list[1:]:
        merged_df = pd.merge(merged_df, df, on='datetime', how='outer', suffixes=('', '_dup'))
elif merged_df is None:
    merged_df = pd.read_csv('merged_df.csv')

In [5]:
# 합친 dataframe에서 중복 컬럼 확인
n = 0
unique_col = []
for l in merged_df.columns.sort_values():
    if 'dup' in l:
        continue
    elif n != 10:
        unique_col.append(l)
        n += 1
    else:
        n = 0
        print()

In [6]:
unique_col

['Unnamed: 0',
 'addresses_count_active',
 'addresses_count_receiver',
 'addresses_count_sender',
 'block_bytes',
 'block_count',
 'block_interval',
 'blockreward',
 'blockreward_usd',
 'close',
 'coinbase_premium_index',
 'datetime',
 'difficulty',
 'fees_block_mean',
 'fees_block_mean_usd',
 'fees_reward_percent',
 'fees_total',
 'fees_total_usd',
 'fees_transaction_mean',
 'fees_transaction_mean_usd',
 'fees_transaction_median_usd',
 'funding_rates',
 'hashrate',
 'long_liquidations',
 'long_liquidations_usd',
 'open_interest',
 'short_liquidations',
 'short_liquidations_usd',
 'supply_new',
 'supply_total',
 'taker_buy_sell_ratio',
 'taker_buy_volume',
 'taker_sell_ratio',
 'taker_sell_volume',
 'tokens_transferred_mean',
 'tokens_transferred_median',
 'tokens_transferred_total',
 'transactions_count_mean',
 'transactions_count_total',
 'utxo_count',
 'volume']

In [1]:
if not os.path.exists('merged_df.csv'):
    merged_df.to_csv('merged_df.csv')

NameError: name 'os' is not defined

In [10]:
'''
관점에 따라 다르겠지만
올해 데이터가 내년의 데이터의 경향을 반영한다고 보긴 쉽지 않다.

따라서 직년 1년의 데이터만 쓰는 게 아니라 
1 ~ 3년 정도의 데이터를 사용하고
1년, 분기, 등으로 나눠서 생각해봐야겠다

'''

'datetime'

,Unnamed: 0,datetime,coinbase_premium_gap,coinbase_premium_index,funding_rates,funding_rates_dup,funding_rates_dup.1,funding_rates_dup.2,funding_rates_dup.3,funding_rates_dup.4,...,hashrate,supply_total,supply_new,tokens_transferred_total,tokens_transferred_mean,tokens_transferred_median,transactions_count_total,transactions_count_mean,utxo_count,velocity_supply_total
0,0,2011-03-13 14:00:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,5634.130000,33.737305,0.010000,NaN,NaN,NaN,NaN
1,1,2011-03-13 15:00:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,3.636053e+02,NaN,NaN,11972.660000,41.002260,7.780000,NaN,NaN,NaN,NaN
2,2,2011-03-13 16:00:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,2.727040e+02,5670800.0,150.0,44161.137793,659.121460,50.000000,NaN,NaN,NaN,NaN
3,3,2011-03-13 17:00:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,3.636053e+02,5671000.0,200.0,5662.831400,123.105030,45.855000,NaN,NaN,NaN,NaN
4,4,2011-03-13 18:00:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,2.727040e+02,5671150.0,150.0,17839.808358,74.332535,0.073196,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
115223,115223,2024-04-26 09:00:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.01,...,7.357868e+11,NaN,NaN,39686.354390,1.758757,0.003737,NaN,NaN,NaN,NaN
115224,115224,2024-04-26 10:00:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.01,...,8.408992e+11,NaN,NaN,32438.993659,1.293318,0.001638,NaN,NaN,NaN,NaN
115225,115225,2024-04-26 11:00:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.01,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
115226,115226,2024-04-26 12:00:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [8]:
merged_df.rename(columns={'datetime':'ID'}, inplace=True)

In [7]:
train_df = pd.read_csv('../../data/train.csv').rename(columns={'target::':'target'})
test_df = pd.read_csv('../../data/test.csv')

train_df, test_df

(                       ID  target
 0     2023-01-01 00:00:00       2
 1     2023-01-01 01:00:00       1
 2     2023-01-01 02:00:00       1
 3     2023-01-01 03:00:00       1
 4     2023-01-01 04:00:00       2
 ...                   ...     ...
 8755  2023-12-31 19:00:00       1
 8756  2023-12-31 20:00:00       1
 8757  2023-12-31 21:00:00       0
 8758  2023-12-31 22:00:00       2
 8759  2023-12-31 23:00:00       2
 
 [8760 rows x 2 columns],
                        ID
 0     2024-01-01 00:00:00
 1     2024-01-01 01:00:00
 2     2024-01-01 02:00:00
 3     2024-01-01 03:00:00
 4     2024-01-01 04:00:00
 ...                   ...
 2787  2024-04-26 03:00:00
 2788  2024-04-26 04:00:00
 2789  2024-04-26 05:00:00
 2790  2024-04-26 06:00:00
 2791  2024-04-26 07:00:00
 
 [2792 rows x 1 columns])

In [31]:
'''merged_df['ID'] = pd.to_datetime(merged_df['ID'])
train_df['ID'] = pd.to_datetime(train_df['ID'])
test_df['ID'] = pd.to_datetime(test_df['ID'])'''

In [12]:
train_merged_df = pd.merge(merged_df, train_df, on='ID', how='inner')
test_merged_df = pd.merge(merged_df, train_df, on='ID', how='inner')

train_merged_df, test_merged_df


(      Unnamed: 0                   ID  coinbase_premium_gap  \
 0         103670  2023-01-01 00:00:00                 -9.86   
 1         103671  2023-01-01 01:00:00                 -8.78   
 2         103672  2023-01-01 02:00:00                 -9.59   
 3         103673  2023-01-01 03:00:00                 -9.74   
 4         103674  2023-01-01 04:00:00                -10.14   
 ...          ...                  ...                   ...   
 8755      112425  2023-12-31 19:00:00                -27.10   
 8756      112426  2023-12-31 20:00:00                -24.73   
 8757      112427  2023-12-31 21:00:00                -28.48   
 8758      112428  2023-12-31 22:00:00                 -9.08   
 8759      112429  2023-12-31 23:00:00                  4.48   
 
       coinbase_premium_index  funding_rates  funding_rates_dup  \
 0                  -0.059650       0.005049           0.010000   
 1                  -0.053047       0.005049           0.010000   
 2                  -0.057952

In [ ]:
'''# test, train 분리
x_train = train_merged_df.drop(columns=['target', 'ID', ])
y_train = train_merged_df['target']

x_test = test_merged_df.drop(columns=['target', 'ID'])
y_test = test_merged_df['target']'''

In [70]:
def train_lgb(
        train_df: pd.DataFrame
        , test_df: pd.DataFrame
        , drop_col: list[str]=['target', 'ID']
        , target_name: str='target'
)-> tuple[pd.DataFrame, pd.DataFrame, pd.DataFrame, pd.DataFrame, lgb.Booster]:
    x_train = train_merged_df.drop(columns=drop_col)
    y_train = train_merged_df[target_name]

    x_test = test_merged_df.drop(columns=drop_col)
    y_test = test_merged_df[target_name]
    params = {
        "boosting_type": "gbdt",
        "objective": "multiclass",
        "metric": "multi_logloss",
        "num_class": 4,
        "num_leaves": 50,
        "learning_rate": 0.05,
        "n_estimators": 30,
        "random_state": 42,
        "verbose": 0,
    }
    
    train_data = lgb.Dataset(x_train.values, label=y_train.values)
    test_data = lgb.Dataset(x_test.values, label=y_test.values, reference=train_data)
    
    return x_train, y_train, x_test, y_test, lgb.train(
        params=params
        , train_set=train_data
        , valid_sets=test_data
    )

x_train, y_train, x_test, y_test, lgb_model = train_lgb(train_merged_df, test_merged_df)


C:\Users\Minari\anaconda3\envs\bProject01\Lib\site-packages\lightgbm\engine.py:172: UserWarning: Found `n_estimators` in params. Will use it instead of argument
  _log_warning(f"Found `{alias}` in params. Will use it instead of argument")


In [72]:
y_test_pred = lgb_model.predict(x_test)
y_test_pred_class = np.argmax(y_test_pred, axis=1)

accuracy = accuracy_score(y_test, y_test_pred_class)
auroc = roc_auc_score(y_test, y_test_pred, multi_class='ovr')

print(f'acc: {accuracy}, auroc: {auroc}')

acc: 0.7571917808219178, auroc: 0.9304804899180017


In [73]:
train_merged_df_with_target = x_train.copy()
train_merged_df_with_target['close'] = y_train

# 피어슨 상관계수 계산
correlation_matrix = train_merged_df_with_target.corr()

# 타겟과 각 피처 간의 상관계수만 추출
target_correlations = correlation_matrix['close'].drop('close').sort_values(ascending=False)

# 상관계수 출력
target_correlations

long_liquidations_usd_dup      0.055130
long_liquidations_dup          0.055063
long_liquidations_dup.3        0.054655
long_liquidations_usd_dup.3    0.054421
long_liquidations_dup.2        0.052132
                                 ...   
taker_buy_volume_dup.6              NaN
taker_sell_volume_dup.6             NaN
taker_buy_ratio_dup.6               NaN
taker_sell_ratio_dup.6              NaN
taker_buy_sell_ratio_dup.6          NaN
Name: close, Length: 252, dtype: float64

In [74]:
f_import = lgb_model.feature_importance(importance_type='gain')
f_names = x_train.columns

gain_df = pd.DataFrame({
    'feature': f_names
    , 'importance': f_import/f_import.sum()
})

gain_df.sort_values(by='importance', ascending=False, inplace=True)

gain_df

,feature,importance
144,open_interest_dup.7,0.039880
178,close,0.031220
201,taker_sell_volume_dup.3,0.026652
191,taker_sell_volume_dup.1,0.025704
181,taker_sell_volume,0.019415
...,...,...
70,long_liquidations_usd_dup.13,0.000000
71,short_liquidations_usd_dup.13,0.000000
166,open_interest_dup.29,0.000000
72,long_liquidations_dup.14,0.000000


In [75]:
# target 값에 관여한 횟수 
split_df = pd.DataFrame({
    'feature': x_train.columns
    , 'importance': lgb_model.feature_importance(importance_type='split')
})
split_df

,feature,importance
0,Unnamed: 0,14
1,coinbase_premium_gap,56
2,coinbase_premium_index,44
3,funding_rates,52
4,funding_rates_dup,46
...,...,...
248,tokens_transferred_median,50
249,transactions_count_total,47
250,transactions_count_mean,86
251,utxo_count,13


In [76]:
# target 값에 관여하지 않은 피처 일단 제거 후 목록 추출
unused_features = split_df[split_df['importance'] == 0]
unused_features

,feature,importance
10,funding_rates_dup.6,0
36,long_liquidations_dup.5,0
37,short_liquidations_dup.5,0
38,long_liquidations_usd_dup.5,0
39,short_liquidations_usd_dup.5,0
...,...,...
219,taker_buy_sell_ratio_dup.6,0
224,taker_buy_sell_ratio_dup.7,0
242,fees_reward_percent,0
244,supply_total,0


In [77]:
# target 값에 관여하지 않은 피처의 gain 값 확인
unused_features_gain = gain_df[gain_df['feature'].isin(unused_features['feature'])]
unused_features_gain

,feature,importance
213,taker_sell_ratio_dup.5,0.0
219,taker_buy_sell_ratio_dup.6,0.0
224,taker_buy_sell_ratio_dup.7,0.0
36,long_liquidations_dup.5,0.0
37,short_liquidations_dup.5,0.0
...,...,...
70,long_liquidations_usd_dup.13,0.0
71,short_liquidations_usd_dup.13,0.0
166,open_interest_dup.29,0.0
72,long_liquidations_dup.14,0.0


In [78]:
# 해당 피처들을 실제 값에서 확인
train_merged_df[unused_features['feature']]
test_merged_df[unused_features['feature']]

,funding_rates_dup.6,long_liquidations_dup.5,short_liquidations_dup.5,long_liquidations_usd_dup.5,short_liquidations_usd_dup.5,long_liquidations_dup.7,short_liquidations_dup.7,long_liquidations_usd_dup.7,short_liquidations_usd_dup.7,long_liquidations_usd_dup.11,...,taker_buy_sell_ratio_dup.5,taker_buy_volume_dup.6,taker_sell_volume_dup.6,taker_buy_ratio_dup.6,taker_sell_ratio_dup.6,taker_buy_sell_ratio_dup.6,taker_buy_sell_ratio_dup.7,fees_reward_percent,supply_total,supply_new
0,NaN,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,...,4.046784,NaN,NaN,NaN,NaN,NaN,0.475845,0.007425,1.924871e+07,75.00
1,NaN,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,...,14.473239,NaN,NaN,NaN,NaN,NaN,0.823286,0.010146,1.924874e+07,25.00
2,NaN,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,...,3.529514,NaN,NaN,NaN,NaN,NaN,4.375441,0.006221,1.924879e+07,50.00
3,NaN,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,...,6.477528,NaN,NaN,NaN,NaN,NaN,0.987453,0.006985,1.924882e+07,31.25
4,NaN,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,...,0.711501,NaN,NaN,NaN,NaN,NaN,1.294601,0.007821,1.924886e+07,43.75
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8755,NaN,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,...,0.919945,NaN,NaN,NaN,NaN,NaN,1.876219,0.265839,1.958605e+07,56.25
8756,NaN,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,...,0.469806,NaN,NaN,NaN,NaN,NaN,1.201989,0.254407,1.958607e+07,25.00
8757,NaN,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,...,1.886957,NaN,NaN,NaN,NaN,NaN,0.876970,0.269433,1.958609e+07,18.75
8758,NaN,0.0,0.0,0.0,0.0,1.869435,0.0,79100.0,0.0,0.0,...,0.775883,NaN,NaN,NaN,NaN,NaN,0.510369,0.296094,1.958611e+07,12.50


In [64]:
# 해당 피처들을 제외한 값을 추출
train_merged_df.drop(columns=unused_features['feature'])
test_merged_df.drop(columns=unused_features['feature'])

,Unnamed: 0,ID,coinbase_premium_gap,coinbase_premium_index,funding_rates,funding_rates_dup,funding_rates_dup.1,funding_rates_dup.2,funding_rates_dup.3,funding_rates_dup.4,...,fees_total_usd,hashrate,tokens_transferred_total,tokens_transferred_mean,tokens_transferred_median,transactions_count_total,transactions_count_mean,utxo_count,velocity_supply_total,target
0,103670,2023-01-01 00:00:00,-9.86,-0.059650,0.005049,0.010000,0.001400,0.010000,0.000571,0.010000,...,9271.503609,5.062917e+11,33057.024011,2.885312,0.020125,11457.0,954.750000,83308092.0,65.978971,2
1,103671,2023-01-01 01:00:00,-8.78,-0.053047,0.005049,0.010000,0.001400,0.010000,0.000570,0.010000,...,4237.095391,1.687639e+11,12933.965951,2.217758,0.021293,5832.0,1458.000000,83314883.0,65.977755,1
2,103672,2023-01-01 02:00:00,-9.59,-0.057952,0.005049,0.010000,0.001400,0.010000,0.000566,0.010000,...,5176.614029,3.375278e+11,26960.250177,4.857703,0.022980,5550.0,693.750000,83314090.0,65.975570,1
3,103673,2023-01-01 03:00:00,-9.74,-0.058912,0.005067,0.010000,0.001518,0.010000,0.000557,0.010000,...,3632.186132,2.109549e+11,32747.924338,6.243646,0.024678,5245.0,1049.000000,83326258.0,65.975092,1
4,103674,2023-01-01 04:00:00,-10.14,-0.061373,0.006210,0.010000,0.008400,0.010000,0.000536,0.010000,...,5699.177070,2.953368e+11,26550.999095,3.824690,0.028596,6942.0,991.714286,83339168.0,65.969825,2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8755,112425,2023-12-31 19:00:00,-27.10,-0.063587,0.036371,0.036288,0.014468,0.024847,0.035893,0.064891,...,868199.890699,7.731601e+11,34918.916733,0.845781,0.000620,41286.0,4587.333333,153112663.0,15.474537,1
8756,112426,2023-12-31 20:00:00,-24.73,-0.058109,0.037233,0.038245,0.018500,0.023523,0.036404,0.064891,...,363206.281009,3.436267e+11,20015.782598,0.927301,0.001052,21585.0,5396.250000,153128637.0,15.474666,1
8757,112427,2023-12-31 21:00:00,-28.48,-0.066979,0.037761,0.040547,0.018500,0.022368,0.038425,0.064891,...,294586.556191,2.577200e+11,13822.498528,1.001413,0.005020,13803.0,4601.000000,153138595.0,15.474537,0
8758,112428,2023-12-31 22:00:00,-9.08,-0.021487,0.038020,0.041638,0.018500,0.021547,0.039708,0.064891,...,223202.409271,1.718134e+11,18699.215727,2.317127,0.009216,8070.0,4035.000000,153156171.0,15.473021,2


In [88]:
# 해당 dataframe으로 학습
x_train, y_train, x_test, y_test, lgb_model = train_lgb(
    train_merged_df.drop(columns=unused_features['feature'])
    , test_merged_df.drop(columns=unused_features['feature'])
)

train_merged_df.drop(columns=unused_features['feature'])

C:\Users\Minari\anaconda3\envs\bProject01\Lib\site-packages\lightgbm\engine.py:172: UserWarning: Found `n_estimators` in params. Will use it instead of argument
  _log_warning(f"Found `{alias}` in params. Will use it instead of argument")


,Unnamed: 0,ID,coinbase_premium_gap,coinbase_premium_index,funding_rates,funding_rates_dup,funding_rates_dup.1,funding_rates_dup.2,funding_rates_dup.3,funding_rates_dup.4,...,fees_total_usd,hashrate,tokens_transferred_total,tokens_transferred_mean,tokens_transferred_median,transactions_count_total,transactions_count_mean,utxo_count,velocity_supply_total,target
0,103670,2023-01-01 00:00:00,-9.86,-0.059650,0.005049,0.010000,0.001400,0.010000,0.000571,0.010000,...,9271.503609,5.062917e+11,33057.024011,2.885312,0.020125,11457.0,954.750000,83308092.0,65.978971,2
1,103671,2023-01-01 01:00:00,-8.78,-0.053047,0.005049,0.010000,0.001400,0.010000,0.000570,0.010000,...,4237.095391,1.687639e+11,12933.965951,2.217758,0.021293,5832.0,1458.000000,83314883.0,65.977755,1
2,103672,2023-01-01 02:00:00,-9.59,-0.057952,0.005049,0.010000,0.001400,0.010000,0.000566,0.010000,...,5176.614029,3.375278e+11,26960.250177,4.857703,0.022980,5550.0,693.750000,83314090.0,65.975570,1
3,103673,2023-01-01 03:00:00,-9.74,-0.058912,0.005067,0.010000,0.001518,0.010000,0.000557,0.010000,...,3632.186132,2.109549e+11,32747.924338,6.243646,0.024678,5245.0,1049.000000,83326258.0,65.975092,1
4,103674,2023-01-01 04:00:00,-10.14,-0.061373,0.006210,0.010000,0.008400,0.010000,0.000536,0.010000,...,5699.177070,2.953368e+11,26550.999095,3.824690,0.028596,6942.0,991.714286,83339168.0,65.969825,2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8755,112425,2023-12-31 19:00:00,-27.10,-0.063587,0.036371,0.036288,0.014468,0.024847,0.035893,0.064891,...,868199.890699,7.731601e+11,34918.916733,0.845781,0.000620,41286.0,4587.333333,153112663.0,15.474537,1
8756,112426,2023-12-31 20:00:00,-24.73,-0.058109,0.037233,0.038245,0.018500,0.023523,0.036404,0.064891,...,363206.281009,3.436267e+11,20015.782598,0.927301,0.001052,21585.0,5396.250000,153128637.0,15.474666,1
8757,112427,2023-12-31 21:00:00,-28.48,-0.066979,0.037761,0.040547,0.018500,0.022368,0.038425,0.064891,...,294586.556191,2.577200e+11,13822.498528,1.001413,0.005020,13803.0,4601.000000,153138595.0,15.474537,0
8758,112428,2023-12-31 22:00:00,-9.08,-0.021487,0.038020,0.041638,0.018500,0.021547,0.039708,0.064891,...,223202.409271,1.718134e+11,18699.215727,2.317127,0.009216,8070.0,4035.000000,153156171.0,15.473021,2


In [87]:
# 추출된 데이터로 학습한 결과
y_test_pred = lgb_model.predict(x_test)
y_test_pred_class = np.argmax(y_test_pred, axis=1)

accuracy = accuracy_score(y_test, y_test_pred_class)
auroc = roc_auc_score(y_test, y_test_pred, multi_class='ovr')

print(f'acc: {accuracy}, auroc: {auroc}')

acc: 0.7571917808219178, auroc: 0.9304804899180017


In [90]:
# 제외된 데이터로 학습
x_train, y_train, x_test, y_test, lgb_model = train_lgb(
    train_merged_df[unused_features['feature']]
    , test_merged_df[unused_features['feature']]
)
test_merged_df[unused_features['feature']]

C:\Users\Minari\anaconda3\envs\bProject01\Lib\site-packages\lightgbm\engine.py:172: UserWarning: Found `n_estimators` in params. Will use it instead of argument
  _log_warning(f"Found `{alias}` in params. Will use it instead of argument")


,funding_rates_dup.6,long_liquidations_dup.5,short_liquidations_dup.5,long_liquidations_usd_dup.5,short_liquidations_usd_dup.5,long_liquidations_dup.7,short_liquidations_dup.7,long_liquidations_usd_dup.7,short_liquidations_usd_dup.7,long_liquidations_usd_dup.11,...,taker_buy_sell_ratio_dup.5,taker_buy_volume_dup.6,taker_sell_volume_dup.6,taker_buy_ratio_dup.6,taker_sell_ratio_dup.6,taker_buy_sell_ratio_dup.6,taker_buy_sell_ratio_dup.7,fees_reward_percent,supply_total,supply_new
0,NaN,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,...,4.046784,NaN,NaN,NaN,NaN,NaN,0.475845,0.007425,1.924871e+07,75.00
1,NaN,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,...,14.473239,NaN,NaN,NaN,NaN,NaN,0.823286,0.010146,1.924874e+07,25.00
2,NaN,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,...,3.529514,NaN,NaN,NaN,NaN,NaN,4.375441,0.006221,1.924879e+07,50.00
3,NaN,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,...,6.477528,NaN,NaN,NaN,NaN,NaN,0.987453,0.006985,1.924882e+07,31.25
4,NaN,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,...,0.711501,NaN,NaN,NaN,NaN,NaN,1.294601,0.007821,1.924886e+07,43.75
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8755,NaN,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,...,0.919945,NaN,NaN,NaN,NaN,NaN,1.876219,0.265839,1.958605e+07,56.25
8756,NaN,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,...,0.469806,NaN,NaN,NaN,NaN,NaN,1.201989,0.254407,1.958607e+07,25.00
8757,NaN,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,...,1.886957,NaN,NaN,NaN,NaN,NaN,0.876970,0.269433,1.958609e+07,18.75
8758,NaN,0.0,0.0,0.0,0.0,1.869435,0.0,79100.0,0.0,0.0,...,0.775883,NaN,NaN,NaN,NaN,NaN,0.510369,0.296094,1.958611e+07,12.50


In [91]:
# 제외 되었던 데이터로 학습한 결과
y_test_pred = lgb_model.predict(x_test)
y_test_pred_class = np.argmax(y_test_pred, axis=1)

accuracy = accuracy_score(y_test, y_test_pred_class)
auroc = roc_auc_score(y_test, y_test_pred, multi_class='ovr')

print(f'acc: {accuracy}, auroc: {auroc}')

acc: 0.7571917808219178, auroc: 0.9304804899180017


In [83]:
acc_cor = pd.Series()

for n, v in target_correlations.items():
    print(n, v)

long_liquidations_usd_dup 0.05513046142628597
long_liquidations_dup 0.05506319962483757
long_liquidations_dup.3 0.05465456351486498
long_liquidations_usd_dup.3 0.05442074641100817
long_liquidations_dup.2 0.05213180104276266
long_liquidations_usd_dup.2 0.051910354400641415
taker_sell_ratio_dup 0.049723874671580764
taker_sell_ratio 0.049466899820600145
long_liquidations 0.04198256650314807
long_liquidations_usd 0.04185487891657136
long_liquidations_dup.23 0.040721756163997896
long_liquidations_dup.20 0.04055395931588894
long_liquidations_dup.1 0.039599078048218404
long_liquidations_usd_dup.1 0.03876304617375404
taker_sell_volume_dup.1 0.0386439395664742
long_liquidations_dup.18 0.03859018356056583
long_liquidations_dup.21 0.03853420928998866
long_liquidations_dup.29 0.038151684879222866
long_liquidations_dup.17 0.037936332294114285
long_liquidations_dup.27 0.037626231946148996
long_liquidations_dup.15 0.03757975026880825
long_liquidations_usd_dup.23 0.036688793714433114
long_liquidations

In [ ]:
merged_df

In [10]:
# 각 분류 단위 별로 피처명 혹은 데이터프레임 분리
interval = 'HOURLY'
categories = [
    {'MARKET-DATA': ['FUNDING-RATES', 'LIQUIDATIONS']} 
    , {'NETWORK-DATA': []}
]


file_names: List[str] = [
    f for f in os.listdir(data_path) if f.startswith(interval) and f.endswith('.csv')
]

for file in file_names:
    print(file)
    # print(file.split('_')[2])

HOURLY_MARKET-DATA_COINBASE-PREMIUM-INDEX.csv
HOURLY_MARKET-DATA_FUNDING-RATES_ALL_EXCHANGE.csv
HOURLY_MARKET-DATA_FUNDING-RATES_BINANCE.csv
HOURLY_MARKET-DATA_FUNDING-RATES_BITMEX.csv
HOURLY_MARKET-DATA_FUNDING-RATES_BYBIT.csv
HOURLY_MARKET-DATA_FUNDING-RATES_DERIBIT.csv
HOURLY_MARKET-DATA_FUNDING-RATES_HTX_GLOBAL.csv
HOURLY_MARKET-DATA_FUNDING-RATES_HUOBI_GLOBAL.csv
HOURLY_MARKET-DATA_FUNDING-RATES_OKEX.csv
HOURLY_MARKET-DATA_FUNDING-RATES_OKX.csv
HOURLY_MARKET-DATA_LIQUIDATIONS_ALL_EXCHANGE_ALL_SYMBOL.csv
HOURLY_MARKET-DATA_LIQUIDATIONS_BINANCE_ALL_SYMBOL.csv
HOURLY_MARKET-DATA_LIQUIDATIONS_BINANCE_BTC_BUSD.csv
HOURLY_MARKET-DATA_LIQUIDATIONS_BINANCE_BTC_USD.csv
HOURLY_MARKET-DATA_LIQUIDATIONS_BINANCE_BTC_USDT.csv
HOURLY_MARKET-DATA_LIQUIDATIONS_BITFINEX_ALL_SYMBOL.csv
HOURLY_MARKET-DATA_LIQUIDATIONS_BITFINEX_BTC_USDT.csv
HOURLY_MARKET-DATA_LIQUIDATIONS_BITMEX_ALL_SYMBOL.csv
HOURLY_MARKET-DATA_LIQUIDATIONS_BITMEX_BTC_USD.csv
HOURLY_MARKET-DATA_LIQUIDATIONS_BYBIT_ALL_SYMBOL.csv
HOURL